In [ ]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
import os.path
dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
dir_data_clim='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/clim/'
def sss_filename(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    fname_tem=syr + '/'+ 'mercatorglorys12v1_gl12_mean_' + syr + smon + '.nc'
    filename = dir_data + fname_tem
    return filename
def sss_filename_new(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    fname_tem=syr + '/'+ 'subset_mercatorglorys12v1_gl12_mean_' + syr + smon + '.nc'
    filename = dir_data + fname_tem
    return filename
def sss_filename_new_yearly(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    fname_tem=syr + '/'+ 'year_subset_mercatorglorys12v1_gl12_mean_' + syr + '.nc'
    filename = dir_data + fname_tem
    return filename

def sss_nrt_filename(d):
    dir_data='F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    d2 = d + dt.timedelta(days = +2)
    syr2, smon2, sdym2 =str(d2.year).zfill(4), str(d2.month).zfill(2), str(d2.day).zfill(2)
    fname_tem = 'metoffice_coupled_orca025_GL4_SAL_b' + syr2 + smon2 + sdym2 + '_dm' + syr + smon + sdym + '.nc'
    filename = dir_data + fname_tem
    return filename
def ssh_nrt_filename(d):
    dir_data='F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    d2 = d + dt.timedelta(days = +2)
    syr2, smon2, sdym2 =str(d2.year).zfill(4), str(d2.month).zfill(2), str(d2.day).zfill(2)
    fname_tem = 'metoffice_coupled_orca025_GL4_SSH_b' + syr2 + smon2 + sdym2 + '_dm' + syr + smon + sdym + '.nc'
    filename = dir_data + fname_tem
    return filename
def sss_nrt_filename_new(d):
    dir_data='F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    fname_tem = 'monthly/monthly_subset_metoffice_coupled_orca025_GL4_SAL_b' + syr + smon + '_dm20180208.nc'
    filename = dir_data + fname_tem
    return filename
def sss_nrt_filename_new_yearly(d):
    dir_data='F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/'
    syr, smon, sdym =str(d.year).zfill(4), str(d.month).zfill(2), str(d.day).zfill(2)
    fname_tem = 'monthly/year_subset_metoffice_coupled_orca025_GL4_SAL_b' + syr + '_dm20180208.nc'
    filename = dir_data + fname_tem
    return filename

#get 0.25 km grid from cmc data to downsamplt this
filename = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/1994/002/19940102120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])
ds_v2.close()

In [ ]:
#MONTHLY files are huge becauseo of depth layers, so lower spatial resolution and only save surface data
for lyr in range(1999,2018): #2017):
    for imon in range(1,13): #3,13):
        d = dt.date(lyr,imon,1) 
        filename = sss_filename(d)
        if os.path.exists(filename):      
            ds = xr.open_dataset(filename)
            ds_subset = ds.sel(depth = ds.depth[0].data)
            ds_low_res = ds_subset.interp(latitude = new_lat,longitude = new_lon)
            ds.close()
            filename_new = sss_filename_new(d)
            ds_low_res.to_netcdf(filename_new)
            os.remove(filename)

In [ ]:
lyr, idyjl = 2018, 1
d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
filename = sss_nrt_filename(d)
print(filename)
ds = xr.open_dataset(filename)
ds.close()
ds = ds.mean('depth')
ds
filename = ssh_nrt_filename(d)
print(filename)
ds2 = xr.open_dataset(filename)
ds2.close()
ds2
ds['zos']=ds2['zos']
ds

In [ ]:
#Daily NRT files are huge becauseo of depth layers, so lower spatial resolution and only save surface data
#also put into monthly file          
file1 = 'F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/2017/year_subset_mercatorglorys12v1_gl12_mean_2017.nc'
ds1 = xr.open_dataset(file1)
ds1.close()
for lyr in range(2018,2019): 
    ds_mnth=[]
    for imon in range(1,13): #:13):
        init = 0
        for idyjl in range(1,366):
            d = dt.date(lyr,1,1) + dt.timedelta(idyjl - 1)
            dd=dt.datetime(lyr,1,1) + dt.timedelta(idyjl - 1)
            if d.month!=imon:
                continue
            filename = sss_nrt_filename(d)
            ds = xr.open_dataset(filename)
            ds = ds.drop('time').mean('time')
            ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon').sortby('lat')
            #ds['time']=dd
            ds.assign_coords(time=dd)
            ds.expand_dims('time')
            ds_subset = ds.sel(depth = ds.depth[0].data)
            ds.close()
            filename = ssh_nrt_filename(d)
            ds_tem = xr.open_dataset(filename)
            ds_tem = ds_tem.drop('time').mean('time')
            ds_tem = ds_tem.assign_coords(lon=(((ds_tem.lon + 180) % 360) - 180)).sortby('lon').sortby('lat')
            #ds_tem['time']=dd
            ds_tem.assign_coords(time=dd)
            ds_tem.expand_dims('time')
            ds_subset['zos']=ds_tem['zos']
            ds_tem.close()
            ds_low_res = ds_subset.interp(lat = new_lat,lon = new_lon)
            print(ds_low_res)
            if init==0:
                ds_sum = ds_low_res
                init = 1
            else:
                ds_sum = xr.concat([ds_sum,ds_low_res],dim = 'time')
            print(idyjl,ds_sum.dims)
        #ds_clim2 = ds_sum.resample(time='M').mean()   
        ds_clim2 = ds_sum.mean('time')
        #ds_clim2.assign_coords(time=dd.month)
        ds_clim2.expand_dims('time',0)
        dd=dt.datetime(lyr,imon,1)
        ds_clim2.coords['time']=ds1.time[imon-1].values+np.timedelta64(365,'D')
#        ds_clim2 = ds_sum.groupby('time.month').mean('time')
        #ds_sum = ds_sum.mean('time',skipna=True)
        ds_mnth.append(ds_clim2)
    combined = xr.concat(ds_mnth, dim='time')
    filename_new = sss_nrt_filename_new_yearly(d)
    combined.to_netcdf(filename_new)
    

In [ ]:
file1 = 'F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/2017/year_subset_mercatorglorys12v1_gl12_mean_2017.nc'
ds1 = xr.open_dataset(file1)
ds1.close()
print(ds1.time[0])
print(ds1.time[0].values+np.timedelta64(days=365))

In [ ]:
print(filename_new)
print(combined)
combined.so[0,:,:].plot()

In [ ]:
#make monthly average ssts into one yearly file
for lyr in range(2017,2018): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        d = dt.date(lyr,imon,1) 
        filename = sss_filename_new(d)
        filename_new = sss_filename_new_yearly(d)
        ds = xr.open_dataset(filename)
        ds.close()
        ds_mnth.append(ds)
    combined = xr.concat(ds_mnth, dim='time')
    combined = combined.rename({'longitude':'lon','latitude':'lat'})
    combined.to_netcdf(filename_new)
    

In [ ]:
#test nrt and reanalysis yearly files
#file1 = 'F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/1993/year_subset_mercatorglorys12v1_gl12_mean_1993.nc'
file2 = 'F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/monthly/year_subset_metoffice_coupled_orca025_GL4_SAL_b2018_dm20180208.nc'
file2a = 'F:/data/model_data/CMEM/global-analysis-forecast-phys_001_015/monthly/year_subset_metoffice_coupled_orca025_GL4_SAL_b2018_dm20180208a.nc'
#ds1 = xr.open_dataset(file1)
ds2 = xr.open_dataset(file2)
#ds2['time']=ds2.time.values+np.timedelta64(365,'D')
#ds1.close()
ds2.close()
#print(ds2)
#print(ds1)
print(ds2)
ds2.to_netcdf(file2a)
#print(dt.timedelta(days=25))
#print(ds1.time[0]+dt.timedelta(days=25))
#d=ds1['time'].values
#d[0].time

In [ ]:
ds2

In [ ]:
print(ds2.time[0].values,ds2.time[0].values+np.timedelta64(days=365))

ds2

In [ ]:
ds1.so[0,:,:].plot(vmin=30,vmax=35)


In [ ]:
ds2.so[0,:,:].plot(vmin=30,vmax=35)


In [ ]:
#calculate climatology
for icase in range(0,3):
    if icase==0:
        iyr1,iyr2 = 1993,2000
    if icase==1:
        iyr1,iyr2 = 2000,2010
    if icase==2:
        iyr1,iyr2 = 2010,2018
    init = 0
    for lyr in range(iyr1,iyr2): 
        d = dt.date(lyr,1,1) 
        filename = sss_filename_new_yearly(d)
        if lyr == 2018:
            filename = sss_nrt_filename_new_yearly(d)
        ds = xr.open_dataset(filename)
        if init==0:
            ds_sum = ds
            init = 1
        else:
            ds_sum = xr.concat([ds_sum,ds],dim = 'time')
        print(lyr,ds_sum.dims)
    ds_sum2 = ds_sum.groupby('time.month').mean('time')
    fname_tem=dir_data_clim + 'climatology_'+str(iyr1)+'_'+str(iyr2-1)+'_mercatorglorys12v1_gl12_mean.nc'
    ds_sum2.to_netcdf(fname_tem)



In [ ]:
num_year = 2017-1993+1
num_year_file1 = 1999 - 1993 + 1
num_year_file2 = 2009 - 2000 + 1
num_year_file3 = 2017 - 2010 + 1
frac_file1 = num_year_file1 / num_year
frac_file2 = num_year_file2 / num_year
frac_file3 = num_year_file3 / num_year
print(frac_file1+frac_file2+frac_file3,frac_file1,frac_file2,frac_file3)
fname_tem = dir_data_clim + 'climatology_1993_1999_mercatorglorys12v1_gl12_mean.nc'
print(fname_tem)
ds = xr.open_dataset(fname_tem)
ds.close()
fname_tem = dir_data_clim + 'climatology_2000_2009_mercatorglorys12v1_gl12_mean.nc'
print(fname_tem)
ds2 = xr.open_dataset(fname_tem)
ds2.close()
fname_tem = dir_data_clim + 'climatology_2010_2017_mercatorglorys12v1_gl12_mean.nc'
print(fname_tem)
ds3 = xr.open_dataset(fname_tem)
ds3.close()
ds_ave = frac_file1*ds + frac_file2*ds2 + frac_file3*ds3
fname_tem = dir_data_clim + 'climatology_1993_2017_mercatorglorys12v1_gl12_mean.nc'
print(fname_tem)
ds_ave.to_netcdf(fname_tem)



In [ ]:
ds_ave

In [ ]:
#put clim together into one 
fname_tem='climatology_monthly_1993_1998_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = ds
ds.close()
fname_tem='climatology_monthly_1999_2004_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'month')
ds.close()
fname_tem='climatology_monthly_2005_2010_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'month')
ds.close()
fname_tem='climatology_monthly_2011_2017_mercatorglorys12v1_gl12_mean.nc'
filename = dir_data_clim + fname_tem
ds = xr.open_dataset(filename)
ds_sum = xr.concat([ds_sum,ds],dim = 'month')
ds.close()
ds_sum2 = ds_sum.groupby('month').mean('month')
#ds_sum2.rename({'longitude':'lon','latitude':'lat'}, inplace = True)
fname_tem='climatology_monthly_1993_2017_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds_sum2.to_netcdf(filename_out)


In [ ]:
ds_sum2

In [ ]:
#change clim to lat and lon rather than latitude and longitude
fname_tem='climatology_monthly_1993_2016_mercatorglorys12v1_gl12_mean.nc'
filename_out = dir_data_clim + fname_tem
ds=xr.open_dataset(filename_out)
#ds.rename({'longitude':'lon','latitude':'lat'}, inplace = True)
print(ds)
#ds.to_netcdf(filename_out)
ds.close()


In [ ]:
ds_sum